In [ ]:
# Module Installation

!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
import tiktoken

In [ ]:
# OpenAI API key
api_key = "YOUR_OPENAI_API_KEY"            # Go on "https://platform.openai.com/api-keys" and get Your OPENAI_API_KEY

In [ ]:
llm_model = "gpt-3.5-turbo"

## 📑 Data Reading

In [ ]:
txt_file_path = './Data/Science.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()

In [ ]:
data

## ✂️ Text Splitting

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [ ]:
data

In [ ]:
len(data)

## 👨‍💻 Embedding Convertion

In [ ]:
# Create vector store
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
vectorstore = FAISS.from_documents(data, embedding=embeddings)

# 🔗 Create conversation chain

In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name=llm_model, openai_api_key=api_key)
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [ ]:
query = "What is a Chemical Reactions and Equations ?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer